In [109]:
from elasticsearch_dsl import connections, Index, Search
from elasticsearch_dsl.query import Range, Terms, Term
from elasticsearch_dsl.aggs import Terms as PerTerms, Cardinality
from json import loads, load
from pandas import DataFrame, NA
from pathlib import Path
from favicon import get as get_favicon
from requests import get

In [ ]:
ELASTIC_HOST = "https://elasticsearch.srv.webis.de"
ELASTIC_PORT = 9200
ELASTIC_USER = "ajjxp"
# As a way to hide the password at least from the notebook, enter a path to a file here, which only contains the password for Elastic.
ELASTIC_PASSWORD_FILE = Path("~/.local/share/passwords/webis-elasticsearch.txt").expanduser()
INDEX = "corpus_mastodon_statuses*"

INSTANCE_DATA_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/fedi_data/2024-01-30/05.jsonl")
INSTANCES_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/04/instances.txt")
REMOVED_INSTANCES_PATH = Path("/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-ernst/sample/04/instances_removed_for_crawling_errors.json")

OUTPUT_PATH = Path("../data")

NUM_EXPLICIT_INSTANCES = 10
# Upper bound for the number of buckets in Elastic aggs. Set a bit higher than the number of instances crawled.
# SEARCH_MAX = 1100
SEARCH_MAX = 10000

# Limit the Elastic searches to a specific date range. Crawling started on 2023-12-21.
DATE_AFTER = "2023-12-01T00:00:00"
## Ca. "2024-01-30T12:00:00" is the time when a new version of the fediverse data was gahtered.
DATE_BEFORE = "2024-02-22T00:00:00"

In [ ]:
# Connect to Elastic.
with ELASTIC_PASSWORD_FILE.open("r") as f:
    password = f.readline().strip("\n")
elastic = connections.create_connection(
    hosts=ELASTIC_HOST + ":" + str(ELASTIC_PORT),
    basic_auth=(ELASTIC_USER, password),
    timeout=300
)

In [ ]:
# Prepare the date query
date_query = Range(crawled_at={"gte": DATE_AFTER, "lte": DATE_BEFORE, "format" : "date_hour_minute_second"})
date_query.to_dict()

In [85]:
base_search: Search = Index(INDEX)\
    .search()\
    .filter(date_query)
base_search.count()

733422562

In [86]:
crawled_instances_count_search: Search = base_search.params(size=0)
crawled_instances_count_search.aggs\
    .metric(
        "instances", 
        Cardinality(field="crawled_from_instance.keyword")
    )
crawled_instances_count = crawled_instances_count_search.execute()\
    .aggs.instances.value
crawled_instances_count

1015

In [195]:
crawled_counts_search: Search = base_search.params(size=0)
crawled_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_counts_buckets = crawled_counts_search.execute().aggs.instances.buckets
crawled_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_counts_buckets
}
crawled_counts

{'mastodon.social': 15668157,
 'mastodon.online': 9857093,
 'mstdn.social': 9638803,
 'ohai.social': 8754882,
 'mas.to': 8208704,
 'mastodon.world': 8149459,
 'universeodon.com': 7489453,
 'social.vivaldi.net': 7194763,
 'techhub.social': 7121898,
 'toot.community': 6660323,
 'convo.casa': 6446080,
 'mastodonapp.uk': 6324971,
 'flipboard.social': 6310298,
 'cupoftea.social': 5642647,
 'masto.ai': 5383642,
 'mastodon.sdf.org': 5241825,
 'hachyderm.io': 5167978,
 'fosstodon.org': 5140898,
 'vmst.io': 5140272,
 'infosec.exchange': 4950838,
 'fedibird.com': 4936551,
 'veganism.social': 4882563,
 'c.im': 4507321,
 'qoto.org': 4506541,
 'mstdn.party': 4430716,
 'mastodon.org.uk': 4395694,
 'chaosfem.tw': 4342564,
 'troet.cafe': 4271040,
 'sfba.social': 4035500,
 'mstdn.nijist.info': 3874928,
 'mstdn.business': 3801452,
 'chaos.social': 3751356,
 'mindly.social': 3746077,
 'mastodon.tokyo': 3643814,
 'fairy.id': 3629844,
 'expressional.social': 3613162,
 'aus.social': 3593440,
 'mastodon.scot

In [ ]:
crawled_federated_counts_search: Search = base_search.params(size=0)\
    .filter(Term(is_local=False))
crawled_federated_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_federated_counts_buckets = crawled_federated_counts_search.execute().aggs.instances.buckets
crawled_federated_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_federated_counts_buckets
}
crawled_federated_counts

In [ ]:
crawled_local_counts_search: Search = base_search.params(size=0)\
    .filter(Term(is_local=True))
crawled_local_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="crawled_from_instance.keyword", size=crawled_instances_count)
    )
    
crawled_local_counts_buckets = crawled_local_counts_search.execute().aggs.instances.buckets
crawled_local_counts = {
    bucket.key: bucket.doc_count
    for bucket in crawled_local_counts_buckets
}
crawled_local_counts

In [160]:
def save_favicon(instance: str, quiet: bool = False) -> Path | None:
    favicon_path = OUTPUT_PATH / "favicons" / f"favicon-{instance}.png"
    favicon_path.parent.mkdir(parents=True, exist_ok=True)
    if favicon_path.exists():
        return favicon_path
    domain = instance
    if instance.count(".") > 1:
        domain = ".".join(instance.split(".")[-2:])
    if not quiet:
        print(f"Searching for favicon for {domain}...")
    try:
        favicons = get_favicon(f"https://{domain}", timeout=2)
    except Exception:
        return None
    favicons = sorted(favicons, key=lambda f: f.width, reverse=True)
    favicons = [favicon for favicon in favicons if favicon.format == "png"]
    if len(favicons) == 0:
        return None
    if not quiet:
        print(f"Downloading favicon from {domain}...")
    favicon_response = get(favicons[0].url, stream=True, timeout=3)
    try:
        with favicon_path.open("wb") as image:
            for chunk in favicon_response.iter_content(1024):
                image.write(chunk)
    except Exception:
        favicon_path.unlink()
        return None
    return favicon_path

In [ ]:
top10_crawled_instances: list[str] = [
    instance
    for instance, _ in sorted(
        crawled_counts.items(), 
        key=lambda b: b[1],
        reverse=True,
    )[:10]
]
top10_crawled_instances

In [117]:
crawled_instances = set()
with INSTANCES_PATH.open("r") as file:
    crawled_instances.update(line.strip() for line in file)
with REMOVED_INSTANCES_PATH.open("rb") as file:
    crawled_instances.update(load(file))
crawled_instances

{'avgeek.social',
 'chaotic.social',
 'oslo.town',
 'assemblag.es',
 'poliversity.it',
 'muenchen.social',
 'sfba.social',
 'mountains.social',
 'mastodonsweden.se',
 'tuiter.rocks',
 'spezi.social',
 'genealysis.social',
 'twit.social',
 'blockriot.com',
 'tabletop.vip',
 'social.vivaldi.net',
 'kurage.cc',
 'lviv.social',
 'mstdn.osaka',
 'iosdev.space',
 'nerdculture.de',
 'mastodon.energy',
 'dizl.de',
 'literatur.social',
 'piaille.fr',
 'tombraidersocial.com',
 'enshittification.social',
 'bangdream.space',
 'awscommunity.social',
 'kemonodon.club',
 'mastodon.design',
 'mindly.social',
 'toots.matapacos.dog',
 '60jj.masto.host',
 'zug.network',
 'metrobus.masto.host',
 'tooting.ch',
 'nederland.online',
 'xno.social',
 'ika.queloud.net',
 'scicomm.xyz',
 'ioc.exchange',
 'mast.hpc.social',
 'sciences.social',
 'friends.cafe',
 'solarcene.community',
 'w3c.social',
 'mastodon.jalgi.eus',
 'tooot.im',
 'parfait.day',
 'saptodon.org',
 '23.illuminati.org',
 'linke.social',
 'alaska

In [118]:
# "Available" data: the entire Mastodon network (that we know of).
## File to dict
available_local_posts = {}
with INSTANCE_DATA_PATH.open("r") as file:
    for line in file:
        line_dict = loads(line)
        # if line_dict["instance"] not in crawled_instances:
        #     continue
        if "nodeinfo" not in line_dict or line_dict["nodeinfo"] is None:
            continue
        nodeinfo = line_dict["nodeinfo"]
        if "usage" not in nodeinfo or nodeinfo["usage"] is None:
            continue
        usage = nodeinfo["usage"]
        if "localPosts" not in usage or usage["localPosts"] is None:
            continue
        available_local_posts[line_dict["instance"]] = usage["localPosts"]
available_local_posts

{'0rb.it': 860,
 '0ab.uk': 14,
 '0v0.ch': 46,
 '0x4655.co.uk': 0,
 '0xee.io': 199,
 '0x50.ch': 2,
 '10mb.social': 475,
 '100terres.social': 4,
 '0371.uk': 152,
 '0x3c.pl': 2340,
 '1210.nl': 958,
 '103.social': 252,
 '10211104.fedicity.net': 26529,
 '13bells.com': 9489,
 '1435.es': 39,
 '1745.info': 316,
 '15-15-15.social': 392,
 '19marbles.org': 4065,
 '1password.social': 913,
 '184.im': 8,
 '1234.as': 767493,
 '23.illuminati.org': 1569,
 '1uca.com': 219815,
 '285936586.masto.host': 16671,
 '27origin.com': 231,
 '0taku.xyz': 705,
 '2pxnl.de': 1484,
 '280562.com': 115,
 '2c.taoetc.org': 1501,
 '333thats33s.cc': 581,
 '3615.computer': 9511,
 '30d.org': 2,
 '3rdstone.cc': 3178,
 '3dp.chat': 7113,
 '3zi.ru': 135897,
 '3rdwall.net': 36,
 '386898.xyz': 45,
 '366.koyomi.online': 2789,
 '39sounds.net': 2034,
 '41020.social': 7032,
 '338338.xyz': 981,
 '433.world': 12315,
 '4four.org': 97,
 '404.inexist.club': 22468,
 '430022.xyz': 12,
 '4458644.xyz': 24,
 '4bear.com': 101372,
 '503junk.house':

In [142]:
len(available_local_posts)

10817

In [143]:
crawled_data = []
for instance in crawled_counts.keys():
    if instance in top10_crawled_instances:
        favicon_path = save_favicon(instance)
    else:
        favicon_path = None
    crawled_data.append({
        "instance": instance,
        "icon": favicon_path if favicon_path is not None else None,
        "posts_available_local": available_local_posts.get(instance, 0),
        "posts_crawled": crawled_counts.get(instance, 0),
        "posts_crawled_federated": crawled_federated_counts.get(instance, 0),
        "posts_crawled_local": crawled_local_counts.get(instance, 0),
    })

df_crawled = DataFrame(crawled_data)
df_crawled["percentage_crawled_federated_of_crawled"] = \
    df_crawled["posts_crawled_federated"] / df_crawled["posts_crawled"]
df_crawled["percentage_crawled_local_of_crawled"] = \
    df_crawled["posts_crawled_local"] / df_crawled["posts_crawled"]
df_crawled["percentage_crawled_of_crawled"] = \
    df_crawled["percentage_crawled_federated_of_crawled"] + \
        df_crawled["percentage_crawled_local_of_crawled"]  # Sanity check.
assert (df_crawled["percentage_crawled_of_crawled"] == 1).all()
df_crawled.drop(columns=["percentage_crawled_of_crawled"])
df_crawled["percentage_crawled_federated_of_all_crawled_federated"] = \
    df_crawled["posts_crawled_federated"] / \
        df_crawled["posts_crawled_federated"].sum()
df_crawled["percentage_crawled_local_of_all_crawled_local"] = \
    df_crawled["posts_crawled_local"] / \
        df_crawled["posts_crawled_local"].sum()
df_crawled["percentage_crawled_of_all_crawled"] = \
    df_crawled["posts_crawled"] / df_crawled["posts_crawled"].sum()
df_crawled["percentage_available_local_of_all_crawled_available_local"] = \
    df_crawled["posts_available_local"] / \
        df_crawled["posts_available_local"].sum()
df_crawled["percentage_available_local_of_all_discovered_available_local"] = \
    df_crawled["posts_available_local"] / \
        sum(available_local_posts.values())
df_crawled["percentage_crawled_local_of_available_local"] = \
    df_crawled["posts_crawled_local"] / df_crawled["posts_available_local"]
df_crawled.head(n=10).style\
    .format({
        col: "{:,.2%}"
        for col in df_crawled.columns
        if col.startswith("percentage_")
    })

,instance,icon,posts_available_local,posts_crawled,posts_crawled_federated,posts_crawled_local,percentage_crawled_federated_of_crawled,percentage_crawled_local_of_crawled,percentage_crawled_of_crawled,percentage_crawled_federated_of_all_crawled_federated,percentage_crawled_local_of_all_crawled_local,percentage_crawled_of_all_crawled,percentage_available_local_of_all_crawled_available_local,percentage_available_local_of_all_discovered_available_local,percentage_crawled_local_of_available_local
0,mastodon.social,../data/favicons/favicon-mastodon.social.png,81205325,15668157,12542364,3125793,80.05%,19.95%,100.00%,1.73%,31.34%,2.14%,21.05%,9.86%,3.85%
1,mastodon.online,../data/favicons/favicon-mastodon.online.png,7759995,9857093,9570010,287083,97.09%,2.91%,100.00%,1.32%,2.88%,1.34%,2.01%,0.94%,3.70%
2,mstdn.social,../data/favicons/favicon-mstdn.social.png,14989224,9638803,9267685,371118,96.15%,3.85%,100.00%,1.28%,3.72%,1.31%,3.89%,1.82%,2.48%
3,ohai.social,../data/favicons/favicon-ohai.social.png,1180837,8754882,8735250,19632,99.78%,0.22%,100.00%,1.21%,0.20%,1.19%,0.31%,0.14%,1.66%
4,mas.to,../data/favicons/favicon-mas.to.png,7021649,8208704,8034084,174620,97.87%,2.13%,100.00%,1.11%,1.75%,1.12%,1.82%,0.85%,2.49%
5,mastodon.world,../data/favicons/favicon-mastodon.world.png,4912465,8149459,7968251,181208,97.78%,2.22%,100.00%,1.10%,1.82%,1.11%,1.27%,0.60%,3.69%
6,universeodon.com,../data/favicons/favicon-universeodon.com.png,2852641,7489453,7399674,89779,98.80%,1.20%,100.00%,1.02%,0.90%,1.02%,0.74%,0.35%,3.15%
7,social.vivaldi.net,../data/favicons/favicon-social.vivaldi.net.png,1709786,7194763,7055731,139032,98.07%,1.93%,100.00%,0.98%,1.39%,0.98%,0.44%,0.21%,8.13%
8,techhub.social,../data/favicons/favicon-techhub.social.png,1404970,7121898,7047603,74295,98.96%,1.04%,100.00%,0.97%,0.74%,0.97%,0.36%,0.17%,5.29%
9,toot.community,../data/favicons/favicon-toot.community.png,1303738,6660323,6630708,29615,99.56%,0.44%,100.00%,0.92%,0.30%,0.91%,0.34%,0.16%,2.27%


In [192]:
print(r"\begin{tabular}{@{}cl@{\hspace{2em}}rr@{\hspace{1.5em}}rrr@{\hspace{1.5em}}rrrr@{\hspace{2em}}rr@{}}")
print(r"  \toprule")
cols = [
    r"\multicolumn{2}{@{}l}{\textbf{Instance}}",
    r"\multicolumn{9}{c}{\textbf{\iconCrawled~Crawled posts}}",
    r"\multicolumn{2}{c}{\textbf{\iconDiscovered~Available}}",
]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \cmidrule(r{2em}){3-11}\cmidrule{12-13}")
cols = [
    r"",
    r"",
    r"\multicolumn{1}{c}{\iconLocalFederated~Loc.+fed.}",
    r"\multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-down}}\iconLocalFederated}",
    r"\multicolumn{1}{c}{\iconFederated~Fed.}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconFederated}",
    r"\multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-left}}\iconLocalFederated}",
    r"\multicolumn{1}{c}{\iconLocal~Local}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconLocal}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-left}}\iconLocalFederated}",
    r"\multicolumn{1}{c@{\hspace{2em}}}{{\tiny\faIcon{arrow-right}}\iconDiscovered}",
    r"\multicolumn{1}{l}{\iconLocal~Local}",
    r"\multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconCrawled}",
]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \midrule")
df_crawled_head = df_crawled.sort_values("posts_crawled", ascending=False).head(n=10)
for _, row in df_crawled_head.iterrows():
    favicon_path = save_favicon(row["instance"], quiet=True)
    cols = [
        r"\favicon{" + (row["instance"] if favicon_path is not None else "mastodon.social") + r"}",
        r"\instance{" + row["instance"] + r"}",
        f"{row['posts_crawled']:,d}",
        f"{row['percentage_crawled_of_all_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['posts_crawled_federated']:,d}",
        f"{row['percentage_crawled_federated_of_all_crawled_federated']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_federated_of_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['posts_crawled_local']:,d}",
        f"{row['percentage_crawled_local_of_all_crawled_local']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_local_of_crawled']:,.0%}".replace("%", r"\%"),
        f"{row['percentage_crawled_local_of_available_local']:,.0%}".replace("%", r"\%"),
        f"{row['posts_available_local']:,d}",
        f"{row['percentage_available_local_of_all_crawled_available_local']:,.0%}".replace("%", r"\%"),
    ]
    print(r"  " + " & ".join(cols) + r" \\")
df_crawled_tail = df_crawled[~df_crawled.index.isin(df_crawled_head.index)]
cols = [
    r"\iconMore",
    f"{len(df_crawled_tail):,d}" + r" others",
    f"{df_crawled_tail['posts_crawled'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_of_all_crawled'].sum():,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_crawled_federated'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_federated_of_all_crawled_federated'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_federated'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_crawled_local'].sum():,d}",
    f"{df_crawled_tail['percentage_crawled_local_of_all_crawled_local'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled_tail['posts_available_local'].sum():,d}",
    f"{(df_crawled_tail['percentage_available_local_of_all_crawled_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
cols = [r"\secondary{" + col + r"}" for col in cols]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \midrule")
cols = [
    r"\iconCrawled",
    f"{len(df_crawled):,d}" + r" crawled",
    f"{df_crawled['posts_crawled'].sum():,d}",
    f"{df_crawled['percentage_crawled_of_all_crawled'].sum():,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_crawled_federated'].sum():,d}",
    f"{df_crawled['percentage_crawled_federated_of_all_crawled_federated'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_federated'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_crawled_local'].sum():,d}",
    f"{df_crawled['percentage_crawled_local_of_all_crawled_local'].sum():,.0%}".replace("%", r"\%"),
    f"{(df_crawled_tail['posts_crawled_local'].sum() / df_crawled['posts_crawled'].sum()):,.0%}".replace("%", r"\%"),
    f"{(df_crawled['posts_crawled_local'].sum() / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
    f"{df_crawled['posts_available_local'].sum():,d}",
    f"{(df_crawled['percentage_available_local_of_all_crawled_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
print(r"  " + " & ".join(cols) + r" \\")
cols = [
    r"\iconDiscovered",
    r"11,822 discovered",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"---",
    f"{sum(available_local_posts.values()):,d}",
    f"{(sum(available_local_posts.values()) / df_crawled['posts_available_local'].sum()):,.0%}".replace("%", r"\%"),
]
cols = [r"\secondary{" + col + r"}" for col in cols]
print(r"  " + " & ".join(cols) + r" \\")
print(r"  \bottomrule")
print(r"\end{tabular}")

\begin{tabular}{@{}cl@{\hspace{2em}}rr@{\hspace{1.5em}}rrr@{\hspace{1.5em}}rrrr@{\hspace{2em}}rr@{}}
  \toprule
  \multicolumn{2}{@{}l}{\textbf{Instance}} & \multicolumn{9}{c}{\textbf{\iconCrawled~Crawled posts}} & \multicolumn{2}{c}{\textbf{\iconDiscovered~Available}} \\
  \cmidrule(r{2em}){3-11}\cmidrule{12-13}
   &  & \multicolumn{1}{c}{\iconLocalFederated~Loc.+fed.} & \multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-down}}\iconLocalFederated} & \multicolumn{1}{c}{\iconFederated~Fed.} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconFederated} & \multicolumn{1}{c@{\hspace{1.5em}}}{{\tiny\faIcon{arrow-left}}\iconLocalFederated} & \multicolumn{1}{c}{\iconLocal~Local} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconLocal} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-left}}\iconLocalFederated} & \multicolumn{1}{c@{\hspace{2em}}}{{\tiny\faIcon{arrow-right}}\iconDiscovered} & \multicolumn{1}{l}{\iconLocal~Local} & \multicolumn{1}{c}{{\tiny\faIcon{arrow-down}}\iconCrawled} \\
  \

In [197]:
source_instances_count_search: Search = base_search.params(size=0)
source_instances_count_search.aggs\
    .metric(
        "instances", 
        Cardinality(field="instance.keyword")
    )
source_instances_count = source_instances_count_search.execute()\
    .aggs.instances.value
source_instances_count

16655

In [205]:
source_unique_post_counts_search: Search = base_search.params(size=0)
source_unique_post_counts_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="instance.keyword", size=source_instances_count)
    )\
    .metric(
        "uris", 
        Cardinality(field="uri.keyword")
    )
    
source_unique_post_counts_buckets = source_unique_post_counts_search.execute().aggs.instances.buckets
source_unique_post_counts = {
    bucket.key: bucket.uris.value
    for bucket in source_unique_post_counts_buckets
}
source_unique_post_counts

{'mastodon.social': 4274826,
 'mstdn.social': 744583,
 'press.coop': 217370,
 'mastodon.online': 474488,
 'mastodon.world': 319467,
 'fedibird.com': 605355,
 'universeodon.com': 179680,
 'submarin.online': 141630,
 'infosec.exchange': 193183,
 'sportsbots.xyz': 667792,
 'mas.to': 337294,
 'misskey.io': 1797527,
 'live-theater.net': 699158,
 'chaos.social': 254880,
 'rss-mstdn.studiofreesia.com': 205544,
 'botsin.space': 223952,
 'masto.globaleas.org': 309973,
 'misskey.gg': 65127,
 'masto.ai': 131717,
 'mastodonapp.uk': 133624,
 'hachyderm.io': 168019,
 'troet.cafe': 258219,
 'tech.lgbt': 218394,
 'mastodon.art': 131288,
 'mstdn.jp': 839146,
 'pubeurope.com': 67264,
 'vivaldi.net': 165435,
 'mstdn.ca': 156046,
 'mamot.fr': 101052,
 'sfba.social': 107726,
 'mastodon.nl': 154164,
 'channels.im': 156308,
 'fosstodon.org': 122733,
 'kolektiva.social': 128107,
 'beige.party': 117760,
 'pravda.me': 107177,
 'newsie.social': 77888,
 'flipboard.com': 110502,
 'gratefuldread.masto.host': 30822,

In [208]:
top_source_instances: list[str] = [
    instance
    for instance, _ in sorted(
        source_unique_post_counts.items(), 
        key=lambda b: b[1],
        reverse=True,
    )[:10]
]
top_source_instances

['mastodon.social',
 'misskey.io',
 'mstdn.jp',
 'mstdn.social',
 'live-theater.net',
 'sportsbots.xyz',
 'rss-parrot.net',
 'fedibird.com',
 'mastodon.online',
 'misskey-square.net']

In [209]:
source_base_search = base_search\
    .filter(Terms(instance__keyword=top_source_instances))
source_base_search.count()

204094157

In [211]:
source_post_stats_search: Search = source_base_search.params(size=0)
source_post_stats_agg_instance = source_post_stats_search.aggs\
    .bucket(
        "instances",
        PerTerms(field="instance.keyword", size=len(top_source_instances))
    )
source_post_stats_agg_instance\
    .metric(
        "uris", 
        Cardinality(field="uri.keyword")
    )
source_post_stats_agg_instance\
    .bucket(
        "is_local",
        PerTerms(field="is_local")
    )\
    .metric(
        "uris", 
        Cardinality(field="uri.keyword")
    )
source_post_stats_agg_instance\
    .bucket(
        "is_local",
        PerTerms(field="is_local")
    )\
    .metric(
        "uris", 
        Cardinality(field="uri.keyword")
    )
    
source_post_stats = source_post_stats_search.execute().aggs
source_post_stats

{'instances': {'doc_count_error_upper_bound': 0, 'sum_other_...}

In [212]:
source_post_stats.to_dict()

{'instances': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'mastodon.social',
    'doc_count': 117824474,
    'uris': {'value': 4274826},
    'is_local': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0,
       'key_as_string': 'false',
       'doc_count': 114698681,
       'uris': {'value': 4272571}},
      {'key': 1,
       'key_as_string': 'true',
       'doc_count': 3125793,
       'uris': {'value': 3136413}}]}},
   {'key': 'mstdn.social',
    'doc_count': 23639564,
    'uris': {'value': 744583},
    'is_local': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 0,
       'key_as_string': 'false',
       'doc_count': 23268446,
       'uris': {'value': 744583}},
      {'key': 1,
       'key_as_string': 'true',
       'doc_count': 371118,
       'uris': {'value': 368703}}]}},
   {'key': 'mastodon.online',
    'doc_count': 19115512,
    'uris': {'value': 474488},

In [ ]:
## Determine the top 10 instances.
index = [
    bucket
    for bucket, _ in zip(sorted(crawled_federated_counts), range(NUM_EXPLICIT_INSTANCES))
]
for i in range(NUM_EXPLICIT_INSTANCES):
    index.append(buckets[i]["key"])
instances = index.copy()
## Count "others".
others_label = str(len(buckets) - NUM_EXPLICIT_INSTANCES) + " others"
## Count "all crawled".
crawled_label = str(len(buckets)) + " crawled"

index.extend((others_label, crawled_label))
index

In [ ]:
# Evaluate data
fed_posts = {}
fed_users = {}
local_posts = {}
local_users = {}
## Federated and local search results look the same, so we can do the same stuff twice.
for search_result, posts, users in (
    (fed_data_result, fed_posts, fed_users),
    (local_data_result, local_posts, local_users)
):
    ## list → dict
    data_dict = {
        entry["key"]: {
            "doc_count": entry["doc_count"],
            "users": entry["users"]["value"]
        }
        for entry in search_result.aggs["instances"]["buckets"]
    }
    ## top 10
    for instance in instances:
        posts[instance] = data_dict[instance]["doc_count"]
        users[instance] = data_dict[instance]["users"]
        del data_dict[instance]
    ## Others / all crawled
    posts[others_label] = sum(v["doc_count"] for v in data_dict.values())
    posts[crawled_label] = sum(posts.values())
## Local users ~(top 10)
local_users[others_label] = sum(v["users"]["value"] for v in local_data_result.aggs["instances"]["buckets"])
## Local users all crawled
local_users[crawled_label] = sum(local_users.values())

In [ ]:
fed_posts

In [ ]:
fed_users

In [ ]:
local_posts

In [ ]:
local_users

In [ ]:
# Federated users and deduplicated posts is not as easy – we need additional searches for these.
## Prepare Query for instances: ~(top 10)
not_top10_query = Q()
for instance in instances:
    not_top10_query = not_top10_query & ~Q("term", crawled_from_instance=instance)

In [ ]:
# Federated posts values contain duplicates. We want to display one of these values later,
# but the main DataFrame should contain deduplicated values as they are way more meaningful.
fed_crawled_dup_posts = fed_posts[crawled_label]
fed_crawled_dup_posts

In [ ]:
# Federated deduplicated posts, ~(top 10).
dedup_posts_other_search = Index(INDEX).search().params(size=0).query(date_query).query("bool", filter=not_top10_query)
dedup_posts_other_search.aggs.bucket("posts", "cardinality", field="uri.keyword")
dedup_posts_other_result = dedup_posts_other_search.execute()

fed_posts[others_label] = dedup_posts_other_result.aggs["posts"]["value"]
fed_posts[others_label]

In [ ]:
# Federated deduplicated posts, all crawled.
dedup_posts_crawled_search = Index(INDEX).search().params(size=0).query(date_query)
dedup_posts_crawled_search.aggs.bucket("posts", "cardinality", field="uri.keyword")
dedup_posts_crawled_result = dedup_posts_crawled_search.execute()

fed_posts[crawled_label] = dedup_posts_crawled_result.aggs["posts"]["value"]
fed_posts[crawled_label]

In [ ]:
## Federated, ~(top 10)
others_fed_users_search = Index(INDEX).search().params(size=0).query(date_query).query("bool", filter=not_top10_query)
others_fed_users_search.aggs.bucket("users", "cardinality", field="account.handle.keyword")
others_fed_users_result = others_fed_users_search.execute()

fed_users[others_label] = others_fed_users_result.aggs["users"]["value"]
fed_users[others_label]

In [ ]:
## Federated, all crawled
all_fed_users_search = Index(INDEX).search().params(size=0).query(date_query)
all_fed_users_search.aggs.bucket("users", "cardinality", field="account.handle.keyword")
all_fed_users_result = all_fed_users_search.execute()

fed_users[crawled_label] = all_fed_users_result.aggs["users"]["value"]
fed_users[crawled_label]

In [ ]:
# "Available" data: the entire Mastodon network (that we know of).
## Get all instances we ever crawled.
crawled_instances = set()
with INSTANCES_PATH.open("r") as f:
    for line in f:
        crawled_instances.add(line[:-1])
with REMOVED_INSTANCES_PATH.open("r") as f:
    for instance in loads(f.readline()):
        crawled_instances.add(instance)
## File to dict
mastodon_data = {}
with open(INSTANCE_DATA_PATH, "r") as file:
    for line in file:
        line_dict = loads(line)
        # No nodeinfo means server unreachable or other software.
        if (line_dict["nodeinfo"]):
            # We need either (nodeinfo & activity) or (nodeinfo & software==mastodon)
            if (not line_dict["activity"]):
                if (not line_dict["nodeinfo"]["software"]
                    or not line_dict["nodeinfo"]["software"]["name"] == "mastodon"
                ):
                    continue
            # We only use nodeinfo though.
            mastodon_data[line_dict["instance"]] = line_dict["nodeinfo"]

all_label = str(len(mastodon_data)) + " discovered"
index.append(all_label)

In [ ]:
avail_local_posts = {
    others_label: 0
}
avail_local_users = {
    others_label: 0
}
## top 10
for instance in instances:
    avail_local_posts[instance] = mastodon_data[instance]["usage"]["localPosts"]
    avail_local_users[instance] = mastodon_data[instance]["usage"]["users"]["total"]
    crawled_instances.remove(instance)
## Others (crawled, but not top 10)
for instance in crawled_instances:
    # Some instances might not be present in more recent fediverse data which was obtained later.
    if (instance in mastodon_data):
        avail_local_posts[others_label] += mastodon_data[instance]["usage"]["localPosts"]
        avail_local_users[others_label] += mastodon_data[instance]["usage"]["users"]["total"]
## All crawled
avail_local_posts[crawled_label] = sum(avail_local_posts.values())
avail_local_users[crawled_label] = sum(avail_local_users.values())
## All
avail_local_posts[all_label] = sum(v["usage"]["localPosts"] for v in mastodon_data.values())
avail_local_users[all_label] = sum(v["usage"]["users"]["total"] for v in mastodon_data.values())

In [ ]:
# Append NA, otherwise it will be NaN and the whole column will be converted to float.
for dict in (fed_posts, fed_users, local_posts, local_users):
    dict[all_label] = NA

df = DataFrame(
    {
        ("Crawled", "Posts (fed.)"): fed_posts,
        ("Crawled", "Users (fed.)"): fed_users,
        ("Crawled", "Posts (loc.)"): local_posts,
        ("Crawled", "Users (loc.)"): local_users,
        ("Available", "Posts (loc.)"): avail_local_posts,
        ("Available", "Users (loc.)"): avail_local_users
    },
    index=index
)
df.index.name = "Instance"
df

In [ ]:
# Calculate ratio of crawled posts and users
local_posts_ratio = df.loc[:, ("Crawled", "Posts (loc.)")] /  df.loc[:, ("Available", "Posts (loc.)")]
local_users_ratio = df.loc[:, ("Crawled", "Users (loc.)")] /  df.loc[:, ("Available", "Users (loc.)")]

fed_posts_ratio = {
    all_label: fed_posts[crawled_label] / avail_local_posts[all_label]
}
fed_users_ratio = {
    all_label: fed_users[crawled_label] / avail_local_users[all_label]
}

df_ratios = DataFrame(
    {
        ("Crawled ratio", "Posts (fed.)"): fed_posts_ratio,
        ("Crawled ratio", "Users (fed.)"): fed_users_ratio,
        ("Crawled ratio", "Posts (loc.)"): local_posts_ratio,
        ("Crawled ratio", "Users (loc.)"): local_users_ratio
    },
    index=index
).fillna(NA, axis=1)
df_ratios.index.name = "Instance"

In [ ]:
print("Posts crawled total:", fed_crawled_dup_posts)
print("Of which are duplicates:", fed_crawled_dup_posts - fed_posts[crawled_label])
print("Ratio of original posts:", fed_posts[crawled_label] / fed_crawled_dup_posts)

df

The above values are deduplicated, which means no single value contains any duplicates. If you take multiple values from one of the *(fed.)* columns though, there will definitely be an intersection.

The values in the *Available* columns above are from a set time. This means, that the ratios below are inexact and, depending on your chosen timeframe, more or less off.

In [ ]:
df_ratios